In [1]:
! vmtkimageviewer --help


Automatic piping vmtkimageviewer
Parsing options vmtkimageviewer

vmtkimageviewer : display a 3D image
  Input arguments:
   -id Id (str,1); default=0: script id
   -handle Self (self,1): handle to self
   -disabled Disabled (bool,1); default=0: disable execution and
     piping
   -i Image (vtkImageData,1): the input image
   -ifile ImageInputFileName (str,1): filename for the default Image
     reader
   -array ArrayName (str,1): name of the array to display
   -renderer vmtkRenderer (vmtkRenderer,1): external renderer
   -windowlevel WindowLevel (float,2); default=[0.0, 0.0]: the
     window/level for displaying the image
   -display Display (bool,1); default=1: toggle rendering
   -margins Margins (bool,1); default=0: toggle margins for tilting
     image planes
   -textureinterpolation TextureInterpolation (bool,1); default=1:
     toggle interpolation of graylevels on image planes
   -continuouscursor ContinuousCursor (bool,1); default=0: toggle use
     of physical continuous c

In [2]:
import os
import sys

In [4]:
sys.path

['',
 '/Users/rick/miniconda3/envs/vmtkplayground/lib/python36.zip',
 '/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6',
 '/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6/lib-dynload',
 '/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6/site-packages',
 '/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6/site-packages/IPython/extensions',
 '/Users/rick/.ipython']

In [7]:
! vmtkimagereader -ifile ./aorta.mha --pipe vmtkimageviewer


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./aorta.mha
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./aorta.mha
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 0.878906 0.878906 1.500090
Origin 156.445000 24.609400 0.000000
Dimensions 157 393 34
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [1.137778101412438, -0.0, 0.0, -177.99969507546882, -0.0, 1.137778101412438, -0.0, -28

In [8]:
! vmtkimagereader -ifile ./aorta.mha --pipe vmtkmarchingcubes -l 1000 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtkimageviewer


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./aorta.mha
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./aorta.mha
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 0.878906 0.878906 1.500090
Origin 156.445000 24.609400 0.000000
Dimensions 157 393 34
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [1.137778101412438, -0.0, 0.0, -177.99969507546882, -0.0, 1.137778101412438, -0.0, -28